In [1]:
import plotly
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go

def calculateMinutesPerView(file_path):
    df = pd.read_csv(file_path)
    flights = df.visit_waypoints_millis
    # the flight time of the initial maneuver is not taken into account.
    flight_count = flights.dropna().count() - 1
    flight_time_sum = df.visit_waypoints_millis.drop([0]).sum()
    minutes_per_view = (flight_time_sum + df.ltstar_millis.sum() + df.global_exploration_millis.sum() + df.local_exploration_millis.sum() )/ flight_count / 1000 / 60
    return minutes_per_view

import os
folder_path = "/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid"
folder_path = "/home/mfaria/Flying_Octomap_code/src/data/current"
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(folder_path):
    for file in f:
        if "state_machine_execution_times.csv" in file:
            files.append(os.path.join(r, file))

In [2]:
minutes_per_view = pd.DataFrame([calculateMinutesPerView(f) for f in files])
mpv_average = minutes_per_view.mean()
mpv_standarDeviation = minutes_per_view.std()
print "The average minutes per view is {} +- {}".format(mpv_average[0], mpv_standarDeviation[0])
print "The median exploration time is {} bounded by {} and {}".format(minutes_per_view.median()[0],  minutes_per_view.min()[0],  minutes_per_view.max()[0])


The average minutes per view is 1.50018695652 +- nan
The median exploration time is 1.50018695652 bounded by 1.50018695652 and 1.50018695652


In [3]:
def calculateTotalTime(file_path):
    df = pd.read_csv(file_path)
    return (df.visit_waypoints_millis.sum() + df.ltstar_millis.sum() + df.global_exploration_millis.sum() + df.local_exploration_millis.sum() ) / 1000 / 60

In [4]:
minutes_per_view = pd.DataFrame([calculateTotalTime(f) for f in files])
mpv_average = minutes_per_view.mean()
mpv_standarDeviation = minutes_per_view.std()
print "The average is {} +- {}".format(mpv_average[0], mpv_standarDeviation[0])


print "The median exploration time is {} bounded by {} and {}".format(minutes_per_view.median()[0],  minutes_per_view.min()[0],  minutes_per_view.max()[0])



The average is 34.5043 +- nan
The median exploration time is 34.5043 bounded by 34.5043 and 34.5043


In [5]:
all_files = pd.DataFrame(pd.DataFrame(pd.read_csv(f).sum())[0] for f in files)

In [6]:
labels = ['Visit waypoints','Global Exploration','Local Exploration','Path Planner']
values = [all_files.visit_waypoints_millis.sum(), all_files.global_exploration_millis.sum(), all_files.local_exploration_millis.sum(), all_files.ltstar_millis.sum()]
colors = ['#9467bd', '#ff800e', '#ffd6b2', '#2ca02c']
trace = go.Pie(
    labels=labels, 
    values=values,
    hole=0.4,
    textfont=dict(color='#1f77b4'),
    marker=dict(colors=colors, 
                       line=dict(color='#ffffff', width=2)))

layout = go.Layout(
        font=dict(
            family='Courier New, monospace',
            size=18
        ),
        title="Execution time by task over all runs",
        titlefont=dict(
            family='Courier New, monospace',
            size=24,
        )
    )

fig=dict(data=[trace], layout=layout)

plotly.offline.iplot(fig)
#pio.write_image(fig, '/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid/execTime_pie.pdf', scale=2)
pio.write_image(fig, '/home/mfaria/Flying_Octomap_code/src/data/current/execTime_pie.pdf', scale=2)
